<a href="https://colab.research.google.com/github/odmina/misbehaving_armadillos/blob/main/Copy_of_Allen_Visual_Behavior_from_SDK_data_2021-07-12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
The following notebook loads data from single behavioral session (one mouse) and one imaging experiment (just one plane) associated with that session to plot stimulus, licks and reward data and neural activity (one selected neuron). 

# Set up environment and import packages

We have built a package called `mindscope_utilities` which contains some useful convenience functions. The `allenSDK` is a dependency of this package and will be automatically installed when you install `mindscope_utilities` per the instrutions below.

We will first install `mindscope_utilities` into our colab environment by running the commands below. When this cell is complete, click on the `RESTART RUNTIME` button that appears at the end of the output. Note that running this cell will produce a long list of outputs and some error messages. Clicking `RESTART RUNTIME` at the end will resolve these issues. 

You can minimize the cell after you are done to hide the output. 

#### Install necessary packages

In [ ]:
!python -m pip install --upgrade pip
!pip install mindscope_utilities --upgrade

     |████████████████████████████████| 1.6MB 7.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 73 kB 1.3 MB/s 
     |████████████████████████████████| 1.7 MB 17.4 MB/s 
     |████████████████████████████████| 47.4 MB 33 kB/s 
     |████████████████████████████████| 130 kB 42.7 MB/s 
     |████████████████████████████████| 72 kB 672 kB/s 
     |████████████████████████████████| 3.4 MB 45.5 MB/s 
     |████████████████████████████████| 98 kB 8.6 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 668 kB 33.0 MB/s 
     |████████████████████████████████| 129 kB 51.0 MB/s 
     |████████████████████████████████| 149 kB 55.0 MB/s 
     |████████████████████████████████| 4.3 MB 35.5 MB/s 
     |████████████████████████████████| 1.3 MB 21.8 MB/s 
     |████████████████████████████████| 3.4 MB 41.1 MB/s 
     |███████████████████████

#### Next we will import packages we need later in the notebook

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import mindscope_utilities
import mindscope_utilities.visual_behavior_ophys as ophys

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

pd.set_option('display.max_columns', 500)

# Load the session and experiment summary tables

We download the data cache:

- `ophys_session_table` contains data describing imaging sessions - one session can contain more than one imaging experiment (more than one plane), and not all behavioral session are accompanied by imaging. contains metadata describing imaging sessions. If more than one plane was imaged during a session, one ophys session id will be associated multiple *ophys experiment ids*. Each ophys session id will also have a unique *behavior session id*
- `behavior_session_table` contains metadata describing behavioral sessions, which may or may not be during imaging. Behavior session ids that do not have *ophys session ids* were training sessions
- `ophys_experiment_table` contains metadata describing imaging experiments (aka imaging planes). When mesoscope is used, one ophys session may contain up to 8 unique experiments (two visual areas by four imaging depths). Some imaging planes may not be released due to quality control issues, thus each ophys session id is associated with anywhere from one to eight unique *experiment ids*. Ophys experiment ids are unique and do not repeat across sessions. To find the same imaging plane that was matched across multiple sessions, use the *ophys_container_id* column that can be found in both ophys_session_table and ophys_experiment_table

In [ ]:
data_storage_directory = "/temp" # Note: this path must exist on your local drive
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

ophys_session_table.csv: 100%|██████████| 165k/165k [00:00<00:00, 2.35MMB/s]
behavior_session_table.csv: 100%|██████████| 885k/885k [00:00<00:00, 10.3MMB/s]
ophys_experiment_table.csv: 100%|██████████| 336k/336k [00:00<00:00, 2.83MMB/s] 


Then we can access the session and experiment tables directly.

Note that a 'session' is a single behavioral session. Sessions that are performed on the mesoscope will have multiple (up to 8) 'experiments' associated with them, where an experiment is a distinct imaging plane. 

In [ ]:
session_table = cache.get_ophys_session_table()
experiment_table = cache.get_ophys_experiment_table()
behavior_session_table = cache.get_behavior_session_table()

We can then view the contents of the session table. Note that this contains a lot of useful metadata about each session. One of the columns, `ophys_experiment_id` provides a list of the experiments (aka imaging planes) that are associated with each session.

In [ ]:
session_table.head()

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,behavior_session_id,ophys_experiment_id,ophys_container_id,project_code,date_of_acquisition,session_type
ophys_session_id,,,,,,,,,,,,,,,,,,,
951410079,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,0,65.0,0,951520319,"[951980471, 951980473, 951980475, 951980479, 9...","[1018028339, 1018028342, 1018028345, 101802835...",VisualBehaviorMultiscope,2019-09-20 09:45:29.897856,OPHYS_1_images_A
952430817,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,209.0,Sst-IRES-Cre,GCaMP6f,2.0,0,66.0,1,952554548,"[953659743, 953659745, 953659749, 953659752]","[1018028339, 1018028345, 1018028354, 1018028357]",VisualBehaviorMultiscope,2019-09-23 08:13:07.627573,OPHYS_2_images_A_passive
954954402,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,210.0,Sst-IRES-Cre,GCaMP6f,3.0,0,67.0,2,953982960,"[958527464, 958527471, 958527474, 958527479, 9...","[1018028339, 1018028342, 1018028345, 101802835...",VisualBehaviorMultiscope,2019-09-24 16:00:00.000000,OPHYS_3_images_A
955775716,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,212.0,Sst-IRES-Cre,GCaMP6f,3.0,1,68.0,3,956010809,"[956941841, 956941844, 956941846]","[1018028339, 1018028342, 1018028345]",VisualBehaviorMultiscope,2019-09-26 08:44:37.199778,OPHYS_3_images_A
957020350,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,213.0,Sst-IRES-Cre,GCaMP6f,4.0,0,0.0,4,957032492,"[957759562, 957759564, 957759566, 957759570, 9...","[1018028339, 1018028342, 1018028345, 101802835...",VisualBehaviorMultiscope,2019-09-27 08:28:05.415005,OPHYS_4_images_B


The experiment table has one row per experiment. Note that the `ophys_session_id` column links each experiment to its associated session in the session_table.

In [ ]:
experiment_table.head()

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,ophys_session_id,behavior_session_id,ophys_container_id,project_code,imaging_depth,targeted_structure,date_of_acquisition,session_type,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,
951980471,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,0,65.0,0,951410079,951520319,1018028342,VisualBehaviorMultiscope,150,VISp,2019-09-20 09:45:29.897856,OPHYS_1_images_A,1085394308
951980484,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,0,65.0,0,951410079,951520319,1018028351,VisualBehaviorMultiscope,75,VISl,2019-09-20 09:45:29.897856,OPHYS_1_images_A,1085400876
951980473,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,0,65.0,0,951410079,951520319,1018028345,VisualBehaviorMultiscope,225,VISp,2019-09-20 09:45:29.897856,OPHYS_1_images_A,1085400919
951980479,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,0,65.0,0,951410079,951520319,1018028354,VisualBehaviorMultiscope,150,VISl,2019-09-20 09:45:29.897856,OPHYS_1_images_A,1085400977
951980475,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,0,65.0,0,951410079,951520319,1018028339,VisualBehaviorMultiscope,75,VISp,2019-09-20 09:45:29.897856,OPHYS_1_images_A,1085400920


We can also view behavior session: 

In [ ]:
behavior_session_table.head()

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,ophys_session_id,ophys_experiment_id,ophys_container_id,project_code,date_of_acquisition,session_type,file_id
behavior_session_id,,,,,,,,,,,,,,,,,,,,
870987812,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,80.0,Sst-IRES-Cre,GCaMP6f,NaN,0,NaN,0,NaN,NaN,NaN,NaN,2019-05-17 14:46:33.550,TRAINING_0_gratings_autorewards_15min,1.085368e+09
872953842,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,84.0,Sst-IRES-Cre,GCaMP6f,NaN,0,NaN,0,NaN,NaN,NaN,NaN,2019-05-21 13:22:20.613,TRAINING_1_gratings,1.085368e+09
873774125,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,85.0,Sst-IRES-Cre,GCaMP6f,NaN,1,NaN,0,NaN,NaN,NaN,NaN,2019-05-22 13:19:52.005,TRAINING_1_gratings,1.085368e+09
874790261,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,86.0,Sst-IRES-Cre,GCaMP6f,NaN,2,NaN,0,NaN,NaN,NaN,NaN,2019-05-23 13:40:31.405,TRAINING_1_gratings,1.085368e+09
875207834,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,87.0,Sst-IRES-Cre,GCaMP6f,NaN,3,NaN,0,NaN,NaN,NaN,NaN,2019-05-24 11:06:39.332,TRAINING_1_gratings,1.085368e+09


In [ ]:
#this selects one mouse, but it seems unncecessary, becouse we are going to select one session and it is always form one mouse
#mouse_id = 457841
#this_mouse_table = behavior_session_table[behavior_session_table['mouse_id'] == mouse_id]
#this_mouse_table

#some additional code

#behavior_session_table = cache.get_behavior_session_table()

#np.random.seed(5) 
#mouse_id = np.random.choice(behavior_session_table['mouse_id'].unique())
#mouse_id

#this_mouse_table = behavior_session_table.query('mouse_id == @mouse_id').sort_values(by = 'date_of_acquisition')
# note that the following is functionally equivalent if you find the syntax easier to read: 
# this_mouse_table = behavior_session_table[behavior_session_table['mouse_id'] == mouse_id]
#this_mouse_table

#behavior_session_ids = this_mouse_table.index.values
#behavior_session_dict = {}
#for behavior_session_id in behavior_session_ids:
#    behavior_session_dict[behavior_session_id] = cache.get_behavior_session(behavior_session_id)

#behavior_session_table = bc.get_behavior_session_table()

# Load one example session
We are going to select one session from this table, session 854060305. This is a session with Sst-IRES-Cre mouse, which expressed GCaMP6f in Sst+ inhibitory interneurons. There were 6 simultaneously acquired imaging planes for this session. 
We can view metadata for this session as follows:

In [ ]:
ophys_session_id = 957020350.0
session_table.loc[ophys_session_id]

equipment_name                                                                MESO.1
full_genotype                          Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt
mouse_id                                                                      457841
reporter_line                                             Ai148(TIT2L-GC6f-ICL-tTA2)
driver_line                                                           [Sst-IRES-Cre]
sex                                                                                F
age_in_days                                                                      213
cre_line                                                                Sst-IRES-Cre
indicator                                                                    GCaMP6f
session_number                                                                     4
prior_exposures_to_session_type                                                    0
prior_exposures_to_image_set                                     

# Find all associated experiments & select one of them

Each session consists of one or more 'experiments', in which each experiment is a single imaging plane

Each mesoscope session has up to 8 experiments associated with the session. We will load all sessions into a dictionary with the experiment IDs as the keys

The first time that this cell is run, the associated NWB files will be downloaded to your local `data_storage_directory`. Subsequent runs of this cell will be faster since the data will already be cached locally.

In [ ]:
experiments = {} #declares dictionary whith experiment ids as keys and 
ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id'] #selects experiments IDs for a session 
#using pandas .loc method (first selects rows with a given ophys_session_id and then just one column 'ophys_exp..')
ophys_experiment_id = ophys_experiment_ids[0] #we are going to use just the first experiment - can be changed to loop below later
print("Selected the first out of ", len(ophys_experiment_ids), "experiments")
#for ophys_experiment_id in ophys_experiment_ids:
experiments[ophys_experiment_id] = cache.get_behavior_ophys_experiment(ophys_experiment_id)
#print(experiments) #this is going to be just a dictionary key with address in memory

Selected the first out of  7 experiments


## Load neural data into memory


The cell below will load the neural data into memory in the pandas 'tidy' format by iterating over each of the 6 experiments and using some helpful tools from the `visual_behavior_ophys` module of the `mindscope_utilities` package that was imported above as `ophys`. 

It will also include a subset of metadata from `ophys_experiment_table` to facilitate splitting by depth, structure (aka cortical area), cre line (aka cell class), etc.

Note that 'tidy' data means that each row represents only one observation. Observations are stacked vertically. Thus, the `timestamps` colums will repeat for every cell in the dataset.

In [ ]:
neural_data = []

this_experiment = experiments[ophys_experiment_id]
this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)
    
# add some columns with metadata for the experiment
metadata_keys = [
        'ophys_experiment_id',
        'ophys_session_id',
        'behavior_session_id', # we may add it to know the associated behavior session
        #on the other hand, it is already included in the ophys session table
        'targeted_structure',
        'imaging_depth',
        'equipment_name',
        'cre_line',
        'mouse_id',
        'sex',
]

for metadata_key in metadata_keys:
  this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]

# we are using only this_experiment and can modify the code to use data from more than one experiment later :)
        
# append the data for this experiment to a list
#neural_data.append(this_experiment_neural_data)
    
# concatate the list of dataframes into a single dataframe
#neural_data = pd.concat(neural_data)

Here is the data for our experiment

In [ ]:
len(this_experiment_neural_data)

386272

It is so long because has one row for each timestamp for each cell. 

Below are the first 5 entries. Again, note that the `tidy` format means that each row has only one observation, which represents a single GCaMP6 fluorescnce value for a single neuron.

In [ ]:
this_experiment_neural_data.head()

,timestamps,cell_roi_id,cell_specimen_id,dff,events,filtered_events,ophys_experiment_id,ophys_session_id,behavior_session_id,targeted_structure,imaging_depth,equipment_name,cre_line,mouse_id,sex
0,8.72468,1080740882,1086614149,0.341778,0.0,0.0,957759562,957020350,957032492,VISp,150,MESO.1,Sst-IRES-Cre,457841,F
1,8.81788,1080740882,1086614149,0.191750,0.0,0.0,957759562,957020350,957032492,VISp,150,MESO.1,Sst-IRES-Cre,457841,F
2,8.91108,1080740882,1086614149,0.169959,0.0,0.0,957759562,957020350,957032492,VISp,150,MESO.1,Sst-IRES-Cre,457841,F
3,9.00428,1080740882,1086614149,-0.003228,0.0,0.0,957759562,957020350,957032492,VISp,150,MESO.1,Sst-IRES-Cre,457841,F
4,9.09748,1080740882,1086614149,0.019101,0.0,0.0,957759562,957020350,957032492,VISp,150,MESO.1,Sst-IRES-Cre,457841,F


- The `cell_roi_id` column contains unique roi ids for all cells in a given experiment, which do not repeat across ophys sessions. 
- The `cell_specimen_id` column contains unique ids for cells that were matched across ophys sessions. Thus, a cell that was imaged in more than one session has multiple roi ids but one cell specimen id. 

# Find associated behavior data

We are using methods created for BehaviorOphysExperiment listed here: [https://allensdk.readthedocs.io/en/latest/visual_behavior_ophys_nwb.html?highlight=BehaviorOphysExperiment](https://allensdk.readthedocs.io/en/latest/visual_behavior_ophys_nwb.html?highlight=BehaviorOphysExperiment)

We are downloading them for session, not experiment, because they are the same for each experiment. If we use more than one experiment in the future downloading them more than once would be huge waste of time :)

In [ ]:
#we have to get relevant behavior_session_id (for the session selected above)
this_behavior_session_id = session_table.loc[ophys_session_id]['behavior_session_id']
this_behavior_session = cache.get_behavior_session(this_behavior_session_id)
this_behavior_session

##Load task parameters

In [ ]:
this_behavior_session_task_params = this_behavior_session.task_parameters

print(this_behavior_session_task_params)

{'stimulus_duration_sec': 0.25, 'task': 'change detection', 'reward_volume': 0.007, 'response_window_sec': [0.15, 0.75], 'session_type': 'OPHYS_4_images_B', 'stimulus': 'images', 'stimulus_distribution': 'geometric', 'omitted_flash_fraction': 0.05, 'n_stimulus_frames': 69575, 'auto_reward_volume': 0.005, 'blank_duration_sec': [0.5, 0.5]}


"\n# add some columns with metadata for the experiment\n#metadata_keys = [\n        'ophys_experiment_id',\n        'ophys_session_id',\n        'behavior_session_id', # we may add it to know the associated behavior session\n        #on the other hand, it is already included in the ophys session table\n        'targeted_structure',\n        'imaging_depth',\n        'equipment_name',\n        'cre_line',\n        'mouse_id',\n        'sex',\n]\n"

In [ ]:
this_behavior_session_stimulus_pres = this_behavior_session.stimulus_presentations
this_behavior_session_stimulus_pres.head()
#len(this_behavior_session_stimulus_pres)

,duration,end_frame,image_index,image_name,image_set,index,omitted,start_frame,start_time,stop_time,is_change
stimulus_presentations_id,,,,,,,,,,,
0,0.25020,18001.0,0,im000,Natural_Images_Lum_Matched_set_ophys_6_2017.07.14,0,False,17986,308.75313,309.00333,False
1,0.25020,18046.0,0,im000,Natural_Images_Lum_Matched_set_ophys_6_2017.07.14,1,False,18031,309.50374,309.75394,False
2,0.25017,18091.0,0,im000,Natural_Images_Lum_Matched_set_ophys_6_2017.07.14,2,False,18076,310.25441,310.50458,False
3,0.25026,18136.0,0,im000,Natural_Images_Lum_Matched_set_ophys_6_2017.07.14,3,False,18121,311.00496,311.25522,False
4,0.25020,18181.0,0,im000,Natural_Images_Lum_Matched_set_ophys_6_2017.07.14,4,False,18166,311.75558,312.00578,False


In [ ]:
this_behavior_session_stimulus_templates = this_behavior_session.stimulus_templates
this_behavior_session_stimulus_templates.head()

,unwarped,warped
image_name,,
im000,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[122, 122, 123, 125, 126, 127, 128, 129, 130,..."
im106,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[108, 109, 106, 103, 102, 104, 107, 112, 117,..."
im075,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[120, 121, 121, 121, 122, 123, 123, 122, 121,..."
im073,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[120, 120, 118, 116, 116, 119, 121, 120, 117,..."
im045,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[10, 13, 6, 0, 0, 8, 15, 13, 6, 2, 4, 9, 12, ..."


In [ ]:
this_behavior_session_stimulus_timestamps = this_behavior_session.stimulus_timestamps
print(this_behavior_session_stimulus_timestamps)

[   8.74134    8.75801    8.77469 ... 4510.21911 4510.23578 4510.25249]


In [ ]:
this_behavior_session_trials = this_behavior_session.trials
this_behavior_session_trials.head()

,start_time,stop_time,lick_times,reward_time,reward_volume,hit,false_alarm,miss,stimulus_change,aborted,go,catch,auto_rewarded,correct_reject,trial_length,response_time,change_frame,change_time,response_latency,initial_image_name,change_image_name
trials_id,,,,,,,,,,,,,,,,,,,,,
0,308.73646,309.85402,"[309.12011, 309.30359, 309.53711]",NaN,0.000,False,False,False,False,True,False,False,False,False,1.11756,NaN,NaN,NaN,NaN,im000,im000
1,310.23770,318.26089,"[314.37439, 314.50783, 314.64127, 314.79139, 3...",314.14086,0.005,False,False,False,True,False,False,False,True,False,8.02319,314.37439,18300.0,314.026728,0.347662,im000,im031
2,318.49442,319.37846,"[318.6779, 318.86138, 319.06156, 319.39515, 31...",NaN,0.000,False,False,False,False,True,False,False,False,False,0.88404,NaN,NaN,NaN,NaN,im031,im031
3,319.99565,323.19826,"[322.58109, 322.71452, 322.88134, 323.46514, 3...",NaN,0.000,False,False,False,False,True,False,False,False,False,3.20261,NaN,NaN,NaN,NaN,im031,im031
4,323.73205,331.02131,"[327.1515, 327.26826, 327.38503, 327.51846, 32...",326.91797,0.005,False,False,False,True,False,False,False,True,False,7.28926,327.15150,19065.0,326.787158,0.364342,im031,im000


In [ ]:
this_behavior_session_licks = this_behavior_session.licks
this_behavior_session_licks.head()

,timestamps,frame
0,8.80804,4
1,9.34178,36
2,9.52524,47
3,9.69204,57
4,15.83037,425


In [ ]:
this_behavior_session_rewards = this_behavior_session.rewards
this_behavior_session_rewards.head()

,volume,timestamps,autorewarded
0,0.005,314.14086,True
1,0.005,326.91797,True
2,0.005,356.92584,True
3,0.005,390.72011,True
4,0.005,427.50016,True


In [ ]:
this_behavior_session_running_speed = this_behavior_session.running_speed
this_behavior_session_running_speed.head()

,timestamps,speed
0,8.74134,-0.006992
1,8.75801,0.389996
2,8.77469,0.754625
3,8.79138,1.057696
4,8.80804,1.277703


In [ ]:
this_behavior_session_raw_running_speed = this_behavior_session.raw_running_speed
this_behavior_session_raw_running_speed.head()

,timestamps,speed
0,8.74134,NaN
1,8.75801,NaN
2,8.77469,NaN
3,8.79138,3.752878
4,8.80804,1.610856


# TRY TO PLOT BEHAVIOUR DATA

In [ ]:
unique_stimuli = [stimulus for stimulus in this_behavior_session.stimulus_presentations['image_name'].unique()]
colormap = {image_name: sns.color_palette()[image_number] for image_number, image_name in enumerate(np.sort(unique_stimuli))}
this_behavior_session.stimulus_presentations['color'] = this_behavior_session.stimulus_presentations['image_name'].map(lambda image_name: colormap[image_name])

In [ ]:
def plot_running(ax, initial_time, final_time):
    '''
    a simple function to plot running speed between two specified times on a specified axis
    inputs:
        ax: axis on which to plot
        intial_time: initial time to plot from
        final_time: final time to plot to
    '''
    running_sample = this_behavior_session.running_speed.query('timestamps >= @initial_time and timestamps <= @final_time')
    ax.plot(
        running_sample['timestamps'],
        running_sample['speed']
    )

def plot_licks(ax, initial_time, final_time):
    '''
    a simple function to plot licks as dots between two specified times on a specified axis
    inputs:
        ax: axis on which to plot
        intial_time: initial time to plot from
        final_time: final time to plot to
    '''
    licking_sample = this_behavior_session.licks.query('timestamps >= @initial_time and timestamps <= @final_time')
    ax.plot(
        licking_sample['timestamps'],
        np.zeros_like(licking_sample['timestamps']),
        marker = 'o',
        color = 'black',
        linestyle = 'none'
    )
    
def plot_rewards(ax, initial_time, final_time):
    '''
    a simple function to plot rewards between two specified times as blue diamonds on a specified axis
    inputs:
        ax: axis on which to plot
        intial_time: initial time to plot from
        final_time: final time to plot to
    '''
    rewards_sample = this_behavior_session.rewards.query('timestamps >= @initial_time and timestamps <= @final_time')
    ax.plot(
        rewards_sample['timestamps'],
        np.zeros_like(rewards_sample['timestamps']),
        marker = 'd',
        color = 'blue',
        linestyle = 'none',
        markersize = 12,
        alpha = 0.5
    )
    
def plot_stimuli(ax, ti, tf):
    '''
    a simple function to plot stimuli as colored vertical spans on a s
    inputs:
        ax: axis on which to plot
        intial_time: initial time to plot from
        final_time: final time to plot to
    '''
    stimulus_presentations_sample = this_behavior_session.stimulus_presentations.query('stop_time >= @initial_time and start_time <= @final_time')
    for idx, stimulus in stimulus_presentations_sample.iterrows():
        ax.axvspan(stimulus['start_time'], stimulus['stop_time'], color=stimulus['color'], alpha=0.25)

def plot_neural_activity (ax, ti, tf, neural_data, cell_id):
  sampling_vector = np.logical_and((neural_data['timestamps'] >= ti).to_numpy(), (neural_data['timestamps'] <= tf).to_numpy(), (neural_data['cell_roi_id'] == cell_id).to_numpy())
  print(type(sampling_vector))
  neural_activity_sample = neural_data.loc[sampling_vector]
  ax.plot(
     neural_activity_sample['timestamps'], 
     neural_activity_sample['dff']
     )

In [ ]:
initial_time = 400 # initial time for plot, in seconds
final_time = 440 # final time for plot, in seconds
fig, ax = plt.subplots(figsize = (15,5))
plot_neural_activity(ax, initial_time, final_time, this_experiment_neural_data, 1080740882)
plot_licks(ax, initial_time, final_time)
plot_rewards(ax, initial_time, final_time)
plot_stimuli(ax, initial_time, final_time)

ax.legend(['running speed', 'licks', 'rewards'])

ax.set_ylabel('running speed (cm/s)')
ax.set_xlabel('time in session (s)')
ax.set_xlim(initial_time, final_time)
ax.set_title('a short section of the session');